In [1]:
import geopandas as gp
%matplotlib inline
import pandas as pd

In [2]:
import folium
from src.utils.map_vis import geojson_per_row

# Mexico

In [3]:
# https://gist.github.com/xijo/d4bad3953f7b9979dd91
REPLACEMENTS = {
    "â‚¬": "€", "â€š": "‚", "â€ž": "„", "â€¦": "…", "Ë†" : "ˆ",
    "â€¹": "‹", "â€˜": "‘", "â€™": "’", "â€œ": "“", "â€" : "”",
    "â€¢": "•", "â€“": "–", "â€”": "—", "Ëœ" : "˜", "â„¢": "™",
    "â€º": "›", "Å“" : "œ", "Å’" : "Œ", "Å¾" : "ž", "Å¸" : "Ÿ",
    "Å¡" : "š", "Å½" : "Ž", "Â¡" : "¡", "Â¢" : "¢", "Â£" : "£",
    "Â¤" : "¤", "Â¥" : "¥", "Â¦" : "¦", "Â§" : "§", "Â¨" : "¨",
    "Â©" : "©", "Âª" : "ª", "Â«" : "«", "Â¬" : "¬", "Â®" : "®",
    "Â¯" : "¯", "Â°" : "°", "Â±" : "±", "Â²" : "²", "Â³" : "³",
    "Â´" : "´", "Âµ" : "µ", "Â¶" : "¶", "Â·" : "·", "Â¸" : "¸",
    "Â¹" : "¹", "Âº" : "º", "Â»" : "»", "Â¼" : "¼", "Â½" : "½",
    "Â¾" : "¾", "Â¿" : "¿", "Ã€" : "À", "Ã‚" : "Â", "Ãƒ" : "Ã",
    "Ã„" : "Ä", "Ã…" : "Å", "Ã†" : "Æ", "Ã‡" : "Ç", "Ãˆ" : "È",
    "Ã‰" : "É", "ÃŠ" : "Ê", "Ã‹" : "Ë", "ÃŒ" : "Ì", "ÃŽ" : "Î",
    "Ã‘" : "Ñ", "Ã’" : "Ò", "Ã“" : "Ó", "Ã”" : "Ô", "Ã•" : "Õ",
    "Ã–" : "Ö", "Ã—" : "×", "Ã˜" : "Ø", "Ã™" : "Ù", "Ãš" : "Ú",
    "Ã›" : "Û", "Ãœ" : "Ü", "Ãž" : "Þ", "ÃŸ" : "ß", "Ã¡" : "á",
    "Ã¢" : "â", "Ã£" : "ã", "Ã¤" : "ä", "Ã¥" : "å", "Ã¦" : "æ",
    "Ã§" : "ç", "Ã¨" : "è", "Ã©" : "é", "Ãª" : "ê", "Ã«" : "ë",
    "Ã¬" : "ì", "Ã­"  : "í", "Ã®" : "î", "Ã¯" : "ï", "Ã°" : "ð",
    "Ã±" : "ñ", "Ã²" : "ò", "Ã³" : "ó", "Ã´" : "ô", "Ãµ" : "õ",
    "Ã¶" : "ö", "Ã·" : "÷", "Ã¸" : "ø", "Ã¹" : "ù", "Ãº" : "ú",
    "Ã»" : "û", "Ã¼" : "ü", "Ã½" : "ý", "Ã¾" : "þ", "Ã¿" : "ÿ"
  }

In [4]:
# http://www.inegi.org.mx/geo/contenidos/geoestadistica/m_g_0.aspx
shp_urban = gp.read_file('data/mexico/mglu2014v6_2/mglu2015v6_2.shp')
shp_state = gp.read_file('data/mexico/mge2014v6_2/mge2015v6_2.shp')
shp_munic = gp.read_file('data/mexico/mgm2014v6_2/mgm2015v6_2.shp')

In [5]:
shp_urban.loc[3396]

CVE_ENT                                                    23
CVE_MUN                                                   005
CVE_LOC                                                  0001
NOM_LOC                                               CancÃºn
geometry    POLYGON ((-86.83937210783648 21.21085723087657...
Name: 3396, dtype: object

In [6]:
def fix_encoding(s):
    for w, r in REPLACEMENTS.items():
        s = s.replace(w, r)
    return s

In [7]:
shp_urban.NOM_LOC = shp_urban.NOM_LOC.apply(fix_encoding)

In [8]:
shp_all = shp_urban.merge(shp_state.drop('geometry', axis=1)).merge(shp_munic.drop('geometry', axis=1))
# shp_all.drop('geometry', axis=1).to_csv('data/mexico/ent-mun-urban.csv')

## match names in shapefiles and the crime statistics

In [9]:
cities = ['Acapulco, Guerrero', 'Cancún, Quintana Roo', 'Ciudad Juárez, Chihuahua', 'Chihuahua, Chihuahua', 
          'Cuernavaca, Morelos', 'Culiacán, Sinaloa', 'Guadalajara, Jalisco', 'Mexicali, Baja California', 
          'Monterrey, Nuevo León', 'Morelia, Michoacán', 'Nuevo Laredo, Tamaulipas', 'Oaxaca, Oaxaca', 'Tijuana, Baja California', 'Toluca, Estado de México', 'Villahermosa, Tabasco']

rows = []
for city in cities:
    cname, sname = city.split(',')
    sname = sname.strip()
    if cname == 'Ciudad Juárez':
        cname='Juárez'
    if sname == 'Estado de México':
        sname ='México'
    if sname == 'Michoacán':
        sname = 'Michoacán de Ocampo'
    rows.append(shp_all[(shp_all.NOM_LOC.str.contains(cname)) & (shp_all.NOM_ENT==sname)])

pd.concat(rows)

,CVE_ENT,CVE_MUN,CVE_LOC,NOM_LOC,geometry,NOM_ENT,NOM_MUN,concat
742,12,001,0001,Acapulco de Juárez,"POLYGON ((-99.9395455851606 16.88883286234676,...",Guerrero,Acapulco de Juárez,12001
3396,23,005,0001,Cancún,POLYGON ((-86.83937210783648 21.21085723087657...,Quintana Roo,Benito Juárez,23005
419,08,010,0005,Ejido Benito Juárez,POLYGON ((-106.8835978923289 30.16639121458292...,Chihuahua,Buenaventura,08010
454,08,037,0001,Juárez,(POLYGON ((-106.4025488769291 31.5450063380720...,Chihuahua,Juárez,08037
431,08,019,0001,Chihuahua,(POLYGON ((-106.1628344308643 28.5909839381582...,Chihuahua,Chihuahua,08019
2095,17,007,0001,Cuernavaca,POLYGON ((-99.25852772267621 19.00069855369537...,Morelos,Cuernavaca,17007
3515,25,006,0001,Culiacán Rosales,(POLYGON ((-107.420703891971 24.74156052279994...,Sinaloa,Culiacán,25006
1186,14,039,0001,Guadalajara,POLYGON ((-103.3135014151582 20.74374155402955...,Jalisco,Guadalajara,14039
48,02,002,0001,Mexicali,POLYGON ((-115.4003165851423 32.67198258432887...,Baja California,Mexicali,02002
2280,19,039,0001,Monterrey,POLYGON ((-100.4206058097103 25.74422449616361...,Nuevo León,Monterrey,19039


In [12]:
# mannually removed 2 lines from above
idx = [742, 3396, 454, 431, 2095, 3515, 1186, 48, 2280, 1943, 3815, 2373, 71, 1771, 3709]
# shp_all.loc[idx]

In [18]:
pairs = []
for i in range(len(idx)):
    row = shp_all.loc[idx[i]]
    pairs.append({'In crime statistics': cities[i], 'in shapefile': '%s, %s' % (row.NOM_LOC, row.NOM_ENT)})
pd.DataFrame(pairs)

,In crime statistics,in shapefile
0,"Acapulco, Guerrero","Acapulco de Juárez, Guerrero"
1,"Cancún, Quintana Roo","Cancún, Quintana Roo"
2,"Ciudad Juárez, Chihuahua","Juárez, Chihuahua"
3,"Chihuahua, Chihuahua","Chihuahua, Chihuahua"
4,"Cuernavaca, Morelos","Cuernavaca, Morelos"
5,"Culiacán, Sinaloa","Culiacán Rosales, Sinaloa"
6,"Guadalajara, Jalisco","Guadalajara, Jalisco"
7,"Mexicali, Baja California","Mexicali, Baja California"
8,"Monterrey, Nuevo León","Monterrey, Nuevo León"
9,"Morelia, Michoacán","Morelia, Michoacán de Ocampo"


## find the boundary for mexico city

In [9]:
target_states = ('Distrito Federal', 'México')
shp_state[shp_state.NOM_ENT.isin(('Distrito Federal', 'México'))]

,CVE_ENT,NOM_ENT,geometry
8,09,Distrito Federal,POLYGON ((-99.11124113799997 19.56149776900003...
14,15,México,POLYGON ((-99.91236793799999 20.28562618500007...


In [10]:
shp_all[shp_all.CVE_ENT.isin(['09','15'])]

,CVE_ENT,CVE_MUN,CVE_LOC,NOM_LOC,geometry,NOM_ENT,NOM_MUN,concat
495,09,002,0001,Azcapotzalco,POLYGON ((-99.18244684564536 19.50756095367512...,Distrito Federal,Azcapotzalco,09002
496,09,003,0001,Coyoacán,POLYGON ((-99.13983600033112 19.35691759966755...,Distrito Federal,Coyoacán,09003
497,09,004,0001,Cuajimalpa de Morelos,"POLYGON ((-99.25737566235688 19.4011202306142,...",Distrito Federal,Cuajimalpa de Morelos,09004
498,09,004,0020,San Lorenzo Acopilco,POLYGON ((-99.32324335439728 19.34193038517918...,Distrito Federal,Cuajimalpa de Morelos,09004
499,09,005,0001,Gustavo A. Madero,POLYGON ((-99.11124113766834 19.56149776910684...,Distrito Federal,Gustavo A. Madero,09005
500,09,006,0001,Iztacalco,"POLYGON ((-99.05578649235269 19.4221410920378,...",Distrito Federal,Iztacalco,09006
501,09,007,0001,Iztapalapa,"POLYGON ((-99.01832866163841 19.3826088311327,...",Distrito Federal,Iztapalapa,09007
502,09,008,0001,La Magdalena Contreras,(POLYGON ((-99.24445636876635 19.2762142001253...,Distrito Federal,La Magdalena Contreras,09008
503,09,009,0001,Villa Milpa Alta,POLYGON ((-99.02828118532841 19.20528638535842...,Distrito Federal,Milpa Alta,09009
504,09,009,0011,San Antonio Tecómitl,POLYGON ((-98.99717701540692 19.22747256940363...,Distrito Federal,Milpa Alta,09009


In [19]:
# http://www.conapo.gob.mx/es/CONAPO/Datos_Abiertos_Delimitacion_de_Zonas_Metropolitanas
metro = gp.read_file('data/mexico/Mapa_de_delimitacion_de_zonas_metropolitanas_2010/ZM_2010.shp').to_crs(epsg=4326)

In [23]:
some_map = folium.Map(location=[19.381495, -99.139095], zoom_start=10)
geojson_per_row(shp_state[shp_state.CVE_ENT=='09'], 'Distrito Federal', color='yellow',some_map=some_map)
geojson_per_row(shp_munic[shp_munic.CVE_ENT=='09'], 'Distrito Federal munic', color='green',some_map=some_map)

# geojson_per_row(shp_state[shp_state.CVE_ENT=='15'], 'Mexico state', color='green',some_map=some_map)
geojson_per_row(shp_all[shp_all.CVE_ENT.isin(['09'])], 'Mexico related urban', tip_cols=['NOM_ENT', 'NOM_MUN', 'NOM_LOC'], some_map=some_map)

geojson_per_row(metro[metro['NOM_SUN']=='Valle de M‚xico'], 'metro area', tip_cols=['NOM_ENT', 'NOM_MUN', 'NOM_SUN'], color='purple', some_map=some_map)
folium.LayerControl().add_to(some_map)
some_map.save('tmp/where is the mexico city.html')

### decide to go for urban area in Distrito Federal

## draw maps

In [ ]:
some_map = folium.Map(location=[24.062199, -102.870324], zoom_start=6)
for i, row in enumerate(shp_all.loc[idx].itertuples()):
    
    if row.geometry.boundary.type=='MultiLineString':
        lines = row.geometry.boundary
    else:
        lines= [row.geometry.boundary]
    for _,line in enumerate(lines):
        tip = 'In crime statistics: %s <br> in shapefile: Localidades Urbanas: %s, Estatales: %s' %(cities[i], row.NOM_LOC, row.NOM_ENT)
        a = folium.PolyLine(
            locations=[(lat,lon) for lon,lat in line.coords], fill_color='blue', 
            tooltip=tip,
            popup=tip
        )
        a.add_to(some_map)
    
some_map.save('mexico cities.html')

In [12]:
some_map = folium.Map(location=[24.062199, -102.870324], zoom_start=6)
for i, row in enumerate(shp_state.itertuples()):
#     if not row.NOM_ENT in ('Distrito Federal', 'México'):
#         continue
    if row.geometry.boundary.type=='MultiLineString':
        lines = row.geometry.boundary
    else:
        lines= [row.geometry.boundary]
    for _,line in enumerate(lines):
        tip = '%s' %(row.NOM_ENT)
        a = folium.Polygon(
            locations=[(lat,lon) for lon,lat in line.coords], 
            tooltip=tip,
            popup=tip, fill_color='blue'
        )
        a.add_to(some_map)
    
some_map.save('tmp/mexico states.html')